In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(".."))
sys.path.append(os.path.abspath("../backend"))
sys.path.append(os.path.abspath(""))

In [6]:
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from backend.ontology import Graph, OntologyManager, OntologyConfig
from backend.datasetmatcher import DatasetManager
from backend.explorative.explorative_support import GuidanceManager
from backend.explorative.llm_query import LLMQuery
from backend.explorative.topic_init import TopicInitator

from backend.eval_config import (
    DBPEDIA_CONFIGS,
    OMA_CONFIGS,
    UNIPROT_CONFIGS,
    BTO_CONFIGS,
    DNB_CONFIGS,
    YAGO_CONFIGS,
    EvalConfig,
)
base_path = "../data"
onto_path = f"{base_path}/hero-ontology/hereditary_clinical.ttl"


# graph = Graph().parse(onto_path, format="turtle")
# graph.bind("bto", "http://www.semanticweb.org/ontologies/2020/3/bto#")

# store = SPARQLStore(
#     "http://localhost:7200/repositories/dpedia",
#     method="POST_FORM",
#     params={"infer": False, "sameAs": False},
# )

# store = SPARQLStore(
#     "http://localhost:3030/dbpedia/query",
#     method="POST_FORM",
#     params={"infer": False, "sameAs": False},
# )
setup: EvalConfig = YAGO_CONFIGS[-1]
print("Setup for ", setup.name)
store = SPARQLStore(
    setup.sparql_endpoint,
    method="POST_FORM",
    params={"infer": False, "sameAs": False},
    retries=10,
)
graph = Graph(store=store)

config = OntologyConfig()

ontology_manager = OntologyManager(config, graph)
dataset_manager = DatasetManager(ontology_manager)
dataset_manager.initialise(glob_path="data/datasets/ALS/**/*.csv")

# ontology_manager.load_full_graph()


Setup for  Yago 3B
-- Loading and merging datasets
0 datasets loaded


In [7]:

guidance_man = GuidanceManager(
    ontology_manager, conn_str=setup.conn_str, llm_model_id=setup.model_id
)
print(guidance_man.identifier)
llm_query = LLMQuery(guidance_man)
topic_init = TopicInitator(guidance_man)

aeb6b1733d9177a93cd331f3455b7055369baf961bc16902023ad11b8db17a17


In [10]:
topic_init.initate()

Initializing topics


Saving anonymous properties: 0it [00:00, ?it/s]


Enriching classes:  35%|███▌      | 3607/10199 [02:13<04:03, 27.11it/s]


KeyboardInterrupt: 

In [ ]:
topic_init.model_topics()

Loading LLM model NousResearch/Hermes-3-Llama-3.2-3B-GGUF None


ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    yes
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4090, compute capability 8.9, VMM: yes
llama_load_model_from_file: using device CUDA0 (NVIDIA GeForce RTX 4090) - 23528 MiB free
llama_model_loader: loaded meta data with 38 key-value pairs and 255 tensors from /home/bkantz/.cache/huggingface/hub/models--NousResearch--Hermes-3-Llama-3.2-3B-GGUF/snapshots/3cd927095d8cbab12c743f932aa63b6f7bbfa141/./Hermes-3-Llama-3.2-3B.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Hermes 3 Llama 3.2 3b Base Fft Chatml...
llama_mod

In [ ]:
llm_query.initiate(n_queries=10, force=True)